# MAE 271: Lab 2 - Inverted pendulum with vertical base excitation
### Cooper Cook & Joshua Booth

## Introduction

## Code and figures

### Global Parameters
Define global parameters and initial conditions for the system:

In [ ]:
import numpy as np
from typing import Any

# -----------------------------
# Global Parameters
# -----------------------------
params: dict[str, Any] = {}
params["l_arm"] = 1.0   # length of pendulum arm [m]
params["m_mass"] = 1.0  # mass of pendulum mass [kg]
params["g"] = 9.8       # gravitational force [m/s^2]

params["amplitude"] = None # oscillator amplitude [m]
params["frequency"] = None # oscillator frequency [Hz]

# Time
t_start: float = 0
t_end: float = 1
t_increment: float = 0.001

t_span = (t_start, t_end)
t_eval = np.arange(min(t_span), max(t_span)+t_increment, t_increment)

### Derivative Function
To solve this non-linear system of ordinary differential equations, we need to start from an initial state, numerically integrate along the state derivatives to find the next state,
 repeat 1) and 2) as desired

For integration, we will use `scipy.integrate`'s `solve_ivp()`, which takes in a range of time (`t_span`) through which to integrate, a function handle (`func`) that takes in the time and state and returns the derivatives, and the initial condition (`initial`) of the state.

Since we wish to solve this problem using several different parameters, we will define a function `get_func()` that returns a function handle that we can pass to `solve_ivp()`.

In [ ]:
from copy import deepcopy

def get_func(params):
    # Make a copy of the input parameters to ensure the returned function 
    # does not change when the original parameters dictionary changes
    params = deepcopy(params)
    ampl: float = params["amplitude"]
    freq: float = params["frequency"]
    leng: float = params["l_arm"]
    mass: float = params["m_mass"]
    g: float = params["g"]

    # Create derivative calculator function to be used by solve_ivp()
    # state = [px, theta]
    def func(t_vals, state):
        px = state[0]
        theta = state[1]

        # Sine and cosine of theta, for computational speed
        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)

        # y-position of cart and its derivatives
        # Y = ampl*np.sin(freq*t_vals)
        # d_Y = ampl*freq*np.cos(freq*t_vals)
        dd_Y = -ampl*(freq**2)*np.sin(freq*t_vals)

        # State derivatives
        d_theta = px/(leng*cos_theta*mass)
        d_px = mass*g*sin_theta*cos_theta + mass*dd_Y*sin_theta*cos_theta - sin_theta/cos_theta*d_theta*px
        
        return [d_px, d_theta]

    return func

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def solve_problem(amplitude: float, frequency: float):
    